In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

learning_rate = 0.001
training_epochs = 10
batch_size = 100

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True )

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [40]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        
        with tf.variable_scope(self.name):
            # dropout에서 사용
            self.training = tf.placeholder(tf.bool) 
            self.X = tf.placeholder(tf.float32, shape = [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, shape = [None, 10])
            
            # Conv_layer1
            # input image = NumOfImage * ( 28 * 28 * 1)
            conv_layer1 = tf.layers.conv2d(inputs = X_img, filters=32, kernel_size = [3, 3], 
                                        padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pooling_layer1 = tf.layers.max_pooling2d(inputs=conv_layer1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # Drop_Out_Layer
            dropout_layer1 = tf.layers.dropout(inputs=pooling_layer1,
                                         rate=0.3, training=self.training)
            # Conv_Layer2
            conv_layer2 = tf.layers.conv2d(inputs=dropout_layer1, filters=64, kernel_size=[3, 3],
                                        padding="SAME", activation=tf.nn.relu)
            # Pooling_Layer
            pooling_layer2 = tf.layers.max_pooling2d(inputs=conv_layer2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # Drop_Out_Layer
            dropout_layer2 = tf.layers.dropout(inputs=pooling_layer2,
                                        rate=0.3, training=self.training)
            # Conv_layer
            conv_layer3 = tf.layers.conv2d(inputs=dropout_layer2, filters=128, kernel_size=[3, 3],
                                        padding="SAME", activation=tf.nn.relu)
            # Pooling_Layer
            pooling_layer3 = tf.layers.max_pooling2d(inputs=conv_layer3, pool_size=[2, 2],
                                        padding="SAME", strides=2)
            # Drop_Out_Layer
            dropout_layer3 = tf.layers.dropout(inputs=pooling_layer3,
                                        rate=0.3, training=self.training)
            # FC
            flat = tf.reshape(dropout_layer3, [-1, 128 * 4 * 4])
            dense_layer = tf.layers.dense(inputs=flat,
                                        units=625, activation=tf.nn.relu)
            dropout_layer4 = tf.layers.dropout(inputs=dense_layer,
                                        rate=0.5, training=self.training)

            self.logits = tf.layers.dense(inputs=dropout_layer4, units=10)
        
        # cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def predict(self, x_test, keep_prop = 1.0):
        return self.sess.run(self.logits,
            feed_dict={self.X :x_test, self.keep_prob:keep_prop})

    def get_accuracy(self, x_test, y_test, keep_prop = 1.0 ):
        return self.sess.run(self.accuracy,
            feed_dict={self.X : x_test, self.Y : y_test, self.keep_prob : keep_prop})

    def train(self, x_data, y_data, keep_prop = 0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict ={
            self.X : x_data, self.Y : y_data, self.keep_prob:keep_prop})

In [42]:
sess = tf.Session()

models = []
num_models = 2
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

# train
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

ValueError: Fetch argument <tf.Operation 'init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init"
op: "NoOp"
input: "^model1/conv2d/kernel/Assign"
input: "^model1/conv2d/bias/Assign"
input: "^model1/conv2d_1/kernel/Assign"
input: "^model1/conv2d_1/bias/Assign"
input: "^model1/conv2d_2/kernel/Assign"
input: "^model1/conv2d_2/bias/Assign"
input: "^model1/dense/kernel/Assign"
input: "^model1/dense/bias/Assign"
input: "^model1/dense_1/kernel/Assign"
input: "^model1/dense_1/bias/Assign"
input: "^beta1_power/Assign"
input: "^beta2_power/Assign"
input: "^model1/conv2d/kernel/Adam/Assign"
input: "^model1/conv2d/kernel/Adam_1/Assign"
input: "^model1/conv2d/bias/Adam/Assign"
input: "^model1/conv2d/bias/Adam_1/Assign"
input: "^model1/conv2d_1/kernel/Adam/Assign"
input: "^model1/conv2d_1/kernel/Adam_1/Assign"
input: "^model1/conv2d_1/bias/Adam/Assign"
input: "^model1/conv2d_1/bias/Adam_1/Assign"
input: "^model1/conv2d_2/kernel/Adam/Assign"
input: "^model1/conv2d_2/kernel/Adam_1